In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
df = pd.read_pickle(r"C:\Users\dpine\red_electrica\red_electrica\data\df_2014_2025.pickle")

In [4]:
df

,date,value
0,2024-08-31 22:00:00+00:00,25532.0
1,2024-08-31 22:05:00+00:00,25400.0
2,2024-08-31 22:10:00+00:00,25327.0
3,2024-08-31 22:15:00+00:00,25229.0
4,2024-08-31 22:20:00+00:00,25080.0
...,...,...
34856,2024-12-30 22:40:00+00:00,26638.0
34857,2024-12-30 22:45:00+00:00,26572.0
34858,2024-12-30 22:50:00+00:00,26377.0
34859,2024-12-30 22:55:00+00:00,26148.0


In [5]:
df.set_index('date', inplace=True)

In [6]:
# Agrupar por hora y calcular la media
df_hourly = df.resample('H').mean()

C:\Users\dpine\AppData\Local\Temp\ipykernel_24748\3965767037.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.resample('H').mean()


In [7]:
df_hourly = df_hourly['value'].interpolate(method='time')

In [8]:
df_hourly = df_hourly.to_frame()

In [9]:
df_hourly

,value
date,
2024-08-31 22:00:00+00:00,24956.916016
2024-08-31 23:00:00+00:00,23609.500000
2024-09-01 00:00:00+00:00,22617.666016
2024-09-01 01:00:00+00:00,21915.000000
2024-09-01 02:00:00+00:00,21555.000000
...,...
2024-12-30 19:00:00+00:00,32806.250000
2024-12-30 20:00:00+00:00,32516.666016
2024-12-30 21:00:00+00:00,30309.833984


In [10]:
df_hourly.isna().sum()

value    0
dtype: int64

In [13]:
df_hourly.duplicated().sum()

np.int64(0)

In [12]:
df_hourly.drop_duplicates(inplace=True)

In [15]:
df = df_hourly.copy()

In [19]:
df.reset_index(inplace=True)

# Añadimos variables de interés

In [21]:
df["hora"] = df.date.dt.hour

In [22]:
df["dia_semana"] = df.date.dt.weekday

In [23]:
df["mes"] = df.date.dt.month

In [24]:
df["trimestre"] = df.date.dt.quarter

In [25]:
df["año"] = df.date.dt.year

In [26]:
import holidays

es_holidays = holidays.Spain(years=[2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025], observed = True)

df["festivo"] = df["date"].dt.date.isin(es_holidays.keys()).astype(int)

In [27]:
pib = pd.read_excel(r"C:\Users\dpine\red_electrica\red_electrica\data\PIB_trimestral_excel.xlsx")
pib = pib.iloc[8:52,0:2]
dato = pib.loc[pib["Series de resultados detallados"] == "2024T4"]["Unnamed: 1"].values[0] * 1.06
dato = pd.DataFrame({'Series de resultados detallados': ["2025T1"], 'Unnamed: 1': [dato]})  # Nueva fila como DataFrame
pib = pd.concat([pib, dato], ignore_index=True)

df['año_trimestre'] = df['año'].astype(str) + 'T' + df['trimestre'].astype(str)
df = df.merge(pib, how= "left", left_on = "año_trimestre", right_on = "Series de resultados detallados")
df.drop(columns = ["año_trimestre", "Series de resultados detallados"], inplace = True)
df.rename(columns = {"Unnamed: 1":"pib_trimestral"}, inplace = True)

c:\Users\dpine\red_electrica\red_electrica\venv2\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [28]:
df

,date,value,hora,dia_semana,mes,trimestre,año,festivo,pib_trimestral
0,2024-08-31 22:00:00+00:00,24956.916016,22,5,8,3,2024,0,393311
1,2024-08-31 23:00:00+00:00,23609.500000,23,5,8,3,2024,0,393311
2,2024-09-01 00:00:00+00:00,22617.666016,0,6,9,3,2024,0,393311
3,2024-09-01 01:00:00+00:00,21915.000000,1,6,9,3,2024,0,393311
4,2024-09-01 02:00:00+00:00,21555.000000,2,6,9,3,2024,0,393311
...,...,...,...,...,...,...,...,...,...
2871,2024-12-30 19:00:00+00:00,32806.250000,19,0,12,4,2024,0,420435
2872,2024-12-30 20:00:00+00:00,32516.666016,20,0,12,4,2024,0,420435
2873,2024-12-30 21:00:00+00:00,30309.833984,21,0,12,4,2024,0,420435
2874,2024-12-30 22:00:00+00:00,27405.333984,22,0,12,4,2024,0,420435


## Variables de lag

Antes hemos visto que tanto el precio de los productos como el número de ventas de cada uno de ellos, cambiaban de manera bastante frecuente. Vamos a intentar capturar esto para ver como afecta en el número de ventas y el precio en el futuro. Crearemos 4 variables lag de 7 y 15 días, para las variables "item_price" y "item_count"

In [47]:
def crear_lags(df, variable, num_lags = 7):
    
    #Crea el objeto dataframe
    lags = pd.DataFrame()
    
    #Crea todos los lags
    for cada in range(1,num_lags+1):
        lags[variable + '_lag_'+ str(cada)] = df[variable].shift(cada)
    
    #Devuelve el dataframe de lags
    return(lags)

In [48]:
lags_120 = crear_lags(df, variable="value", num_lags=12)

## Variables de medias móviles

En el EDA vimos que el número de ventas por producto era una varaible con mucha varianza. Queremos capturar en una variable la tendencia de esas ventas, y reducir el impacto de la volatilidad de las ventas. Crearemos un mínimo móvil, máximo móvil, y media móvil de 15 días para la variable "item_count", para suavizar la volatilidad de la variable.

In [49]:
def min_movil(df, variable, num_periodos = 7):

    minm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        minm[variable + '_minm_' + str(cada)] = df[variable].shift(1).rolling(cada).min()
    
    #Devuelve el dataframe de lags
    return(minm)

In [50]:
def media_movil(df, variable, num_periodos = 7):

    mm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        mm[variable + '_mm_' + str(cada)] = df[variable].shift(1).rolling(cada).mean()
    
    #Devuelve el dataframe de lags
    return(mm)

In [51]:
def max_movil(df, variable, num_periodos = 7):

    maxm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        maxm[variable + '_maxm_' + str(cada)] = df[variable].shift(1).rolling(cada).max()
    
    #Devuelve el dataframe de lags
    return(maxm)

In [52]:
value_min_movil_df = min_movil(df, variable = 'value', num_periodos=12)

In [53]:
value_mean_movil_df = media_movil(df, variable = 'value', num_periodos= 12)

In [54]:
value_max_movil_df = max_movil(df, variable = 'value', num_periodos= 12)

## Unimos todo

In [58]:
df_todo = pd.concat([df, lags_120, value_min_movil_df, value_mean_movil_df, value_max_movil_df], axis = 1)

In [60]:
df_todo.duplicated().sum()

np.int64(0)

In [63]:
df_todo.dropna(inplace=True)

In [64]:
df_todo

,date,value,hora,dia_semana,mes,trimestre,año,festivo,pib_trimestral,value_lag_1,...,value_maxm_3,value_maxm_4,value_maxm_5,value_maxm_6,value_maxm_7,value_maxm_8,value_maxm_9,value_maxm_10,value_maxm_11,value_maxm_12
12,2024-09-01 10:00:00+00:00,22626.583984,10,6,9,3,2024,0,393311,23035.416016,...,23470.750000,23470.750000,23470.750000,23470.750000,23470.750000,23470.750000,23470.750000,23470.750000,23609.500000,24956.916016
13,2024-09-01 11:00:00+00:00,23675.500000,11,6,9,3,2024,0,393311,22626.583984,...,23470.750000,23470.750000,23470.750000,23470.750000,23470.750000,23470.750000,23470.750000,23470.750000,23470.750000,23609.500000
14,2024-09-01 12:00:00+00:00,25849.916016,12,6,9,3,2024,0,393311,23675.500000,...,23675.500000,23675.500000,23675.500000,23675.500000,23675.500000,23675.500000,23675.500000,23675.500000,23675.500000,23675.500000
15,2024-09-01 13:00:00+00:00,25263.250000,13,6,9,3,2024,0,393311,25849.916016,...,25849.916016,25849.916016,25849.916016,25849.916016,25849.916016,25849.916016,25849.916016,25849.916016,25849.916016,25849.916016
16,2024-09-01 14:00:00+00:00,25105.500000,14,6,9,3,2024,0,393311,25263.250000,...,25849.916016,25849.916016,25849.916016,25849.916016,25849.916016,25849.916016,25849.916016,25849.916016,25849.916016,25849.916016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2871,2024-12-30 19:00:00+00:00,32806.250000,19,0,12,4,2024,0,420435,32269.333984,...,32269.333984,32269.333984,32269.333984,32269.333984,32269.333984,32269.333984,32269.333984,32269.333984,32269.333984,32269.333984
2872,2024-12-30 20:00:00+00:00,32516.666016,20,0,12,4,2024,0,420435,32806.250000,...,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000
2873,2024-12-30 21:00:00+00:00,30309.833984,21,0,12,4,2024,0,420435,32516.666016,...,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000
2874,2024-12-30 22:00:00+00:00,27405.333984,22,0,12,4,2024,0,420435,30309.833984,...,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000,32806.250000


precio electricidad, temperatura cada mes

atipicos, describe()